#### 라이브러리

In [ ]:
!pip install --upgrade openai
!pip install tiktoken


In [ ]:
import os
import openai
from openai import OpenAI
import json
import random
import tiktoken
import numpy as np
from collections import defaultdict
from typing import Optional, Tuple, List, Dict
import logging
import random
from datetime import datetime
import pandas as pd
import itertools
import time
from dataclasses import dataclass

#### API 연결하기

In [ ]:
# 발급받은 API 키 설정
OPENAI_API_KEY = ## openai key
client=OpenAI(api_key=OPENAI_API_KEY)

API의 Role을 정의

In [ ]:
# # api key 동작 여부 확인
# # 프로젝트와는 관계없음
# chat = openai.chat.completions.create(
#   model = "gpt-4o-mini-2024-07-18",
#   messages = [
#     {'role': 'system', 'content': 'You are a friendly assistant who provides information about parks located in Mokdong, Yangcheon-gu, Seoul.'},
#     {'role': 'user', 'content': '목동에 있는 공원에 대해 알려줘.'},
# ]
# )
# print(chat)

ChatCompletion(id='chatcmpl-ANVEiacOywIwk9K0FXKsoskemPQJt', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='목동에는 여러 공원이 있습니다. 특히 다음과 같은 공원이 인기 있습니다:\n\n1. **목동근린공원**: 이 공원은 주민들이 운동을 하거나 산책을 하기 좋은 장소입니다. 다양한 운동시설과 산책로가 마련되어 있어 가족 단위 방문객에게 알맞습니다.\n\n2. **서울 목동 어린이공원**: 어린이들을 위한 놀이시설이 잘 갖춰져 있는 공원으로, 아이들이 안전하게 놀 수 있는 환경입니다. 부모들이 아이들과 함께 즐길 수 있는 공간입니다.\n\n3. **목동문화체육센터 주변 공원**: 체육시설과 함께 잘 조성된 녹지 공간이 있어 운동이나 여가활동을 즐기기에 좋습니다. 자전거 도로나 조깅 코스도 마련되어 있어 많은 사람들이 이용합니다.\n\n각 공원마다 특징이 있으니 방문하실 때 어떤 활동을 하고 싶은지에 따라 선택하시면 좋습니다!', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1730164980, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_f59a81427f', usage=CompletionUsage(completion_tokens=205, prompt_tokens=43, total_tokens=248, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails

#### 데이터셋 로드
**데이터 형식** (jsonl)

    {"messages": [
      {"role": "system", "content": "(챗봇의 역할에 대한 설명)"},
      {"role": "user", "content": "(사용자의 예상 질문)"},
      {"role": "assistant", "content": "(챗봇의 예상 답변)"}
      ]
    }


In [ ]:
# 데이터 로드 후 train/valid dataset으로 8:2 분할
data_path = "/content/drive/MyDrive/DS541/QAdataset.jsonl"
train_path = "/content/drive/MyDrive/DS541/train_dataset.jsonl"
valid_path = "/content/drive/MyDrive/DS541/valid_dataset.jsonl"

# data load
with open(data_path, "r") as f:
    lines = [json.loads(line) for line in f]

# 데이터 랜덤 - 일정하게 하려면 코드 주석 처리
random.shuffle(lines)

# 8:2 비율로 나누기
split_index = int(len(lines) * 0.8)
train_data = lines[:split_index]
valid_data = lines[split_index:]

# 훈련 데이터와 테스트 데이터를 jsonl 형식으로 저장
with open(train_path, "w") as train_file:
    for entry in train_data:
        train_file.write(json.dumps(entry) + "\n")

with open(valid_path, "w") as test_file:
    for entry in valid_data:
        test_file.write(json.dumps(entry) + "\n")

print(f"Train data saved as '{train_path}'")
print(f"Valid data saved as '{valid_path}'")

Train data and valid data saved as '/content/drive/MyDrive/DS541/train_dataset.jsonl' 및 '/content/drive/MyDrive/DS541/valid_dataset.jsonl'


In [ ]:
# data path
data_path = "/content/drive/MyDrive/DS541/QAdataset.jsonl" #데이터경로로 변경

# Load dataset
res = client.files.create(
  file=open(data_path, "rb"),
  purpose="fine-tune"
)

resId = res.id
print(f"trained file id : {resId}")

#### Fine-Tuning

In [ ]:
train_path = "/content/drive/MyDrive/DS541/train_dataset.jsonl"
valid_path = "/content/drive/MyDrive/DS541/valid_dataset.jsonl"

train_file = client.files.create(file=open(train_path, "rb"), purpose="fine-tune")
valid_file = client.files.create(file=open(valid_path, "rb"), purpose="fine-tune")

fine_tuning_job = client.fine_tuning.jobs.create(training_file=train_file.id, validation_file=valid_file.id, model="gpt-4o-mini-2024-07-18")

print(fine_tuning_job)

FineTuningJob(id='ftjob-ioyIXM7ybKMcWIuaahDxr3Fd', created_at=1730165932, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-yuMhbeTHpaas9LlsYMEyeUlF', result_files=[], seed=1526429414, status='validating_files', trained_tokens=None, training_file='file-xyJxuztZL7Ifzcu0BfzA65my', validation_file='file-Ic0IybDWRyc8XiK4A8qihNFU', estimated_finish=None, integrations=[], user_provided_suffix=None)


In [ ]:
# fine-tuning 종료 확인
# succeeded 나와야 성공 - 시간 좀 걸림
fine_tune_results = client.fine_tuning.jobs.retrieve(fine_tuning_job.id)
print(fine_tune_results.status)
if fine_tune_results.error:
    print(fine_tune_results.error)
print(fine_tune_results.finished_at)

succeeded
Error(code=None, message=None, param=None)
1730167281


In [ ]:
print(fine_tune_results.fine_tuned_model)

ft:gpt-4o-mini-2024-07-18:personal::ANVprDdJ


    ft:gpt-4o-mini-2024-07-18:personal::ANVprDdJ